In [1]:
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import functions as sf
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import abs
# seting master("local[*]") enables multicore processing on all available logical cores on your machine
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
airlines_df = spark.read.csv(get_training_filename('airlines.csv'), header=True, inferSchema=True)
airports_df = spark.read.csv(get_training_filename('airports.csv'), header=True, inferSchema=True)
flights_df = spark.read.csv(get_training_filename('flights.csv'), header=True, inferSchema=True)


In [4]:
flights_df = flights_df.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'CANCELLED')

flights_df = flights_df.filter((fn.col('CANCELLED')==0))

flights_df = flights_df.withColumn("Flight_Delayed", fn.when(fn.col("DEPARTURE_DELAY")<15, 0).otherwise(1))

from pyspark.ml.feature import Bucketizer
bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df = bucketizer.setHandleInvalid("keep").transform(flights_df)

from pyspark.sql.functions import udf
from pyspark.sql.types import *

t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df = flights_df.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df= encoder.transform(flights_df)

indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df)
flights_df = indexer.transform(flights_df)

encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df= encoder.transform(flights_df)

flights_df = flights_df.drop('ARRIVAL_DELAY')


In [5]:

training_df, testing_df = flights_df.randomSplit([0.9, 0.1], seed=5)


In [6]:
major_df = training_df.filter(col("Flight_Delayed") == 0)
minor_df = training_df.filter(col("Flight_Delayed") == 1)
ratio = major_df.count()/minor_df.count()
print(ratio)


4.4276492398259215

In [7]:
sampled_majority_df = major_df.sample(False, 1.2/ratio, seed=5)
combined_df_2 = sampled_majority_df.unionAll(minor_df)

In [8]:
combined_df_2.show()

+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+--------------+----------+------------------+----------+-----------------------+
MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|DISTANCE|SCHEDULED_ARRIVAL|CANCELLED|Flight_Delayed|Distance_Bucket|Flight_Distance|Airline_Numeric| Airline_OHE|OA_Numeric|Origin_Airport_OHE|DA_Numeric|Destination_Airport_OHE|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+--------------+----------+------------------+----------+-----------------------+
 1| 1| 4| AA| ABQ| DFW| 725| 14| 569| 1015| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 56.0| (627,[56],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| ATL| DFW| 1755| -1| 731| 1930| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 0.0| (627,[0],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| ATL| DFW| 1900| -2| 731| 2030| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 0.0| (627,[0],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| AUS| DFW| 650| -3| 190| 755| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 34.0| (627,[34],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| AUS| DFW| 1330| -2| 190| 1435| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 34.0| (627,[34],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| AUS| DFW| 1555| 3| 190| 1655| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 34.0| (627,[34],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BDL| MIA| 640| -6| 1194| 1006| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 58.0| (627,[58],[1.0])| 24.0| (628,[24],[1.0])|
 1| 1| 4| AA| BNA| DFW| 725| -1| 631| 940| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 30.0| (627,[30],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BNA| DFW| 920| -5| 631| 1130| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 30.0| (627,[30],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BNA| DFW| 1320| -8| 631| 1525| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 30.0| (627,[30],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BOS| DFW| 1135| -3| 1562| 1500| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BOS| DFW| 1605| -6| 1562| 1940| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BOS| LAX| 2020| 8| 2611| 2355| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 4.0| (628,[4],[1.0])|
 1| 1| 4| AA| BOS| MIA| 515| -5| 1258| 843| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 24.0| (628,[24],[1.0])|
 1| 1| 4| AA| BOS| ORD| 700| -1| 867| 905| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| BOS| ORD| 750| -7| 867| 950| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| BOS| ORD| 1755| -8| 867| 2000| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| CLE| DFW| 650| 3| 1021| 905| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 40.0| (627,[40],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| CMH| DFW| 705| -7| 926| 910| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 51.0| (627,[51],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| DAY| DFW| 1250| -3| 861| 1440| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 101.0| (627,[101],[1.0])| 2.0| (628,[2],[1.0])|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+--------------+----------+------------------+----------+-----------------------+
only showing top 20 rows

In [9]:
testing_df.show()

+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+--------------+----------+------------------+----------+-----------------------+
MONTH|DAY|DAY_OF_WEEK|AIRLINE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_DELAY|DISTANCE|SCHEDULED_ARRIVAL|CANCELLED|Flight_Delayed|Distance_Bucket|Flight_Distance|Airline_Numeric| Airline_OHE|OA_Numeric|Origin_Airport_OHE|DA_Numeric|Destination_Airport_OHE|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+--------------+----------+------------------+----------+-----------------------+
 1| 1| 4| AA| ATL| DFW| 2015| 14| 731| 2140| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 0.0| (627,[0],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| ATL| MIA| 815| -9| 594| 1012| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 0.0| (627,[0],[1.0])| 24.0| (628,[24],[1.0])|
 1| 1| 4| AA| AUS| ORD| 1055| -4| 978| 1330| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 34.0| (627,[34],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| BDL| DFW| 720| 0| 1471| 1040| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 58.0| (627,[58],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BDL| DFW| 1235| -8| 1471| 1550| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 58.0| (627,[58],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BOS| ORD| 1155| -8| 867| 1350| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| BOS| ORD| 1540| -11| 867| 1745| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| BOS| ORD| 1700| -5| 867| 1900| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 13.0| (627,[13],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| BWI| DFW| 655| -8| 1217| 935| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 19.0| (627,[19],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| BWI| MIA| 1309| -9| 946| 1550| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 19.0| (627,[19],[1.0])| 24.0| (628,[24],[1.0])|
 1| 1| 4| AA| CLE| DFW| 1555| -3| 1021| 1805| 0| 0| 2.0| Long| 2.0|(13,[2],[1.0])| 40.0| (627,[40],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| CLT| DFW| 1700| 28| 936| 1855| 0| 1| 1.0| Medium| 2.0|(13,[2],[1.0])| 14.0| (627,[14],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| CLT| MIA| 2210| 0| 650| 13| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 14.0| (627,[14],[1.0])| 24.0| (628,[24],[1.0])|
 1| 1| 4| AA| CLT| ORD| 1610| -5| 599| 1720| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 14.0| (627,[14],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| DAY| DFW| 1730| 34| 861| 1915| 0| 1| 1.0| Medium| 2.0|(13,[2],[1.0])| 101.0| (627,[101],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| DCA| MIA| 659| -6| 919| 944| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 22.0| (627,[22],[1.0])| 24.0| (628,[24],[1.0])|
 1| 1| 4| AA| DCA| ORD| 955| 0| 612| 1105| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 22.0| (627,[22],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| DEN| DFW| 855| -3| 641| 1155| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 3.0| (627,[3],[1.0])| 2.0| (628,[2],[1.0])|
 1| 1| 4| AA| DEN| ORD| 1815| 33| 888| 2135| 0| 1| 1.0| Medium| 2.0|(13,[2],[1.0])| 3.0| (627,[3],[1.0])| 1.0| (628,[1],[1.0])|
 1| 1| 4| AA| DFW| ATL| 1005| -1| 731| 1305| 0| 0| 1.0| Medium| 2.0|(13,[2],[1.0])| 2.0| (627,[2],[1.0])| 0.0| (628,[0],[1.0])|
+-----+---+-----------+-------+--------------+-------------------+-------------------+---------------+--------+-----------------+---------+--------------+---------------+---------------+---------------+--------------+----------+------------------+----------+-----------------------+
only showing top 20 rows

In [10]:
#features = sample.columns
#sample = sample.select(col("Flight_Delayed").alias("label"), *features)

In [11]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [12]:

from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StandardScaler

# features = flights_df.columns
# flights_df = flights_df.select(col("Flight_Delayed").alias("label"), *features)

# training_df, testing_df = flights_df.randomSplit([0.9, 0.1], seed=5)

vectorAssembler = VectorAssembler(inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="unscaled_features")

standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")

In [13]:
from pyspark.ml.classification import LinearSVC

In [14]:
lsvc3 = LinearSVC(featuresCol='features', labelCol='Flight_Delayed',maxIter=20, regParam=0.08)
lsvc_pipeline = Pipeline(stages=[vectorAssembler,standardScaler, lsvc3])
lsvc_model3 = lsvc_pipeline.fit(combined_df_2)

In [15]:
predictions_train = lsvc_model3.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

precision recall f1-score support

 0 0.86 0.72 0.79 467809
 1 0.28 0.49 0.36 105586

 micro avg 0.68 0.68 0.68 573395
 macro avg 0.57 0.61 0.57 573395
weighted avg 0.76 0.68 0.71 573395

Out[15]: array([[338426, 129383],
 [ 54096, 51490]])

In [16]:
Recall0=0.72
Recall1=0.49
Balanced_accuracy = (Recall0+Recall1)/2
Balanced_accuracy

Out[1]: 0.605

In [17]:
lsvc4 = LinearSVC(featuresCol='features', labelCol='Flight_Delayed',threshold=0.35)
lsvc_pipeline = Pipeline(stages=[vectorAssembler,standardScaler, lsvc4])
lsvc_model4 = lsvc_pipeline.fit(combined_df_2)

java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:468)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:395)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:177)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:169)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:268)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:240)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:149)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:768)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:453)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:234)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:140)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:612)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:822)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:254)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1400(ManagedSelector.java:62)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:401)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:360)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:184)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:367)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:782)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:914)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:714)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:232)
	... 11 more

In [18]:
predictions_train = lsvc_model4.transform(testing_df)
y_true = predictions_train.select(['Flight_Delayed']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:468)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:395)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:177)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:169)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:268)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:240)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:149)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:768)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:453)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:234)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:140)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:612)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:822)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:254)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1400(ManagedSelector.java:62)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:401)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:360)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:184)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:367)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:782)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:914)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:714)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:232)
	... 11 more

In [19]:

mce = evaluation.MulticlassClassificationEvaluator(labelCol="Flight_Delayed", metricName="accuracy")
score_accuracy = mce.evaluate(lsvc_model4.transform(testing_df))
print(score_accuracy)

In [20]:
scores = [score_auc, score_Recall, score_accuracy, score_f1, score_Precision]
metricName = ['AUC', 'Recall', 'Accuracy', 'f1', 'Precision']

metric_df = pd.DataFrame(zip(metricName,scores),index=[1,2,3,4,5],columns=['Metric Name', 'Score'])
metric_df